# 項目: 分析英國電商銷售數據

## 分析目標

此數據分析的目的是，根據市場銷售數據，挖掘暢銷產品，以便制定更有效的市場策略來提升營收。

## 簡介

原始資料集記錄了一家英國線上零售公司在2010年12月1日至2011年12月9日期間的所有交易情況，涵蓋了該公司在全球不同國家和地區的業務資料。  
該公司主要銷售涵蓋各個場景的禮品，包括但不限於生日禮品、婚禮紀念品、聖誕禮品等等。  
該公司的客戶群主要包括批發商和個人消費者，其中批發商佔了相當大的比例。

資料每列的含義如下：
- `InvoiceNo`: 發票號碼。 6位數，作為交易的唯一識別碼。 如果這個代碼以字母「c」開頭，表示這筆交易被取消。
- `StockCode`: 產品代碼。 5位數，作為產品的唯一識別碼。
- `Description`: 產品名稱。
- `Quantity`: 產品在交易中的數量。
- `InvoiceDate`: 發票日期和時間。 交易發生的日期和時間。
- `UnitPrice`: 單價。 價格單位為英鎊（£）。
- `CustomerID`: 客戶編號。 5位數，作為客戶的唯一識別碼。
- `Country`: 國家名稱。 客戶所居住的國家的名稱。

## 讀取數據

導入數據所需要的函示庫，並透過Pandas的`read_csv`函數，將原始數據文件 "e_commerce.csv" 裡的數據內容，解析為DataFrame，並賦值給變量`original_data`。

In [11]:
import pandas as pd

In [9]:
original_data = pd.read_csv("e_commerce.csv")

In [10]:
original_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 評估數據

在這部分，我將對前面建立的`original_data`這個 DataFrame 所包含的數據進行評估。評估主要從兩個方面進行結構和內容，及整齊度與乾淨度。  
1. 數據的結構性問題指不符合以下標準
- 每列是一個變量
- 每行是一個觀察值
- 每個單元格是一個值
2. 數據的內容性問題包含以下
- 丟失數據
- 重複數據
- 無效數據

### 評估數據整齊度

In [12]:
original_data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299884,563178,23029,DRAWER KNOB CRACKLE GLAZE GREEN,6,8/12/2011 14:12,1.65,16011.0,United Kingdom
206012,554859,22624,IVORY KITCHEN SCALES,12,5/27/2011 10:16,7.65,14828.0,United Kingdom
190085,553182,21977,PACK OF 60 PINK PAISLEY CAKE CASES,4,5/15/2011 12:21,0.55,17107.0,United Kingdom
457075,575730,21034,REX CASH+CARRY JUMBO SHOPPER,2,11/10/2011 19:53,0.95,14765.0,United Kingdom
314762,564654,23266,SET OF 3 WOODEN STOCKING DECORATION,12,8/26/2011 14:29,1.25,14911.0,EIRE
103321,545060,22173,METAL 4 HOOK HANGER FRENCH CHATEAU,2,2/27/2011 13:47,2.95,15547.0,United Kingdom
304542,563584,22197,POPCORN HOLDER,12,8/17/2011 16:48,0.85,15122.0,United Kingdom
291493,562456,23322,LARGE WHITE HEART OF WICKER,12,8/5/2011 10:39,2.95,15194.0,United Kingdom
70873,542105,22212,FOUR HOOK WHITE LOVEBIRDS,4,1/25/2011 13:35,2.10,12854.0,United Kingdom
417153,572650,23310,BUBBLEGUM RING ASSORTED,36,10/25/2011 12:00,0.42,15506.0,United Kingdom


從抽樣的10行數據來看，數據符合前述標準，故不存在結構性問題。

### 評估數據乾淨度

In [13]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


從輸出結果看來，數據共有541909條觀察值，而`Description`、`CustomerID`變量存在缺失值。  
此外，`InvoiceData`的數據類型應為日期，`CustomerID`的數據類型應為字串，應當進行數據格式轉換。

### 評估缺失數據

在暸解`Description`存在缺失值後，根據條件提取出缺失觀察值。

In [15]:
original_data[original_data["Description"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,12/7/2011 18:26,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,12/7/2011 18:31,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,12/7/2011 18:35,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,12/8/2011 10:33,0.0,NaN,United Kingdom


有1454條交易數據缺失`Description`變量值。  
從輸出結果來看，這些缺失`Description`的交易數據，`UnitPrice`都為0。為了驗證此猜想，我們增加篩選條件，看是否存在`Description`變量缺失且`UnitPrice`不為0的數據。

In [16]:
original_data[(original_data["Description"].isnull()) & (original_data["UnitPrice"] != 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


篩選出來結果數量為0條，說明缺失`Description`值的數據，同時也不具備有效的`UnitPrice`值。  
`Description`表示產品名稱，`UnitPrice`表示產品單價，都是進行後續商品交易分析的重要變量。如果他們同時缺失/無效，我們認為數據無法提供有效含義，因此這些後續可以被刪除。

In [17]:
original_data[original_data["CustomerID"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom


`CustomerID`表示客戶編號，不是分析暢銷商品的必要變量。並且從輸出結果來看，有些`CustomerID`缺失的交易數據仍然有效，因此保留此變量為空的觀察值。

### 評估重複數據

根據數據變量的含義來看，雖然`InvoiceNo`、`StockCode`和`CustomerID`都是唯一標示符，但一次交易可能包含多件商品，因此`InvoiceNo`可存在重複，不同交易可以包含同件商品，因此`StockCode`可以存在重複，顧客可以進行多次交易或下單多個商品，因此`CustomerID`也可以存在重複。故針對此數據集，無需評估重複數據。

### 評估不一致數據

不一致數據可能存在於`Country`變量中，查看是否存在多個不同值只待同一國家的情況。

In [18]:
original_data["Country"].value_counts()

Country
United Kingdom          495266
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
China                      288
Singapore                  229
USA                        218
UK                         211
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United States               73
United Arab Emirates        68


從`Country`變量值來看，`"USA"`、`"United States"`均在表示美國，`"United Kingdom"`、`"UK"`、`"U.K."`均在表示英國，應在清血階段對這些值進行統一，只保留一個指代值。

### 評估無效或錯誤數據

可以透過DataFrame的`describe`方法，對統計資訊進行快速暸解。

In [19]:
original_data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [22]:
original_data[original_data["Quantity"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


從篩選結果來看，`Quantity`變量為負數的觀察值，`InvoiceNo`似乎均以C開頭，表示訂單被取消。  
但此猜測不一定準確，為了驗證此猜想，增加篩選條件，看是否存在`Quantity`變量為負數且`InvoiceNo`不以C開頭的觀察值。

In [23]:
original_data[(original_data["Quantity"] < 0) & (original_data["InvoiceNo"].str[0] != "C")]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,12/1/2010 16:50,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,12/2/2010 14:42,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,12/3/2010 15:30,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,12/7/2011 18:36,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,12/7/2011 18:38,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,12/7/2011 18:38,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,12/8/2011 9:56,0.0,NaN,United Kingdom


以上猜想被證實錯誤，因此還存在`Quantity`變量為負數且`InvoiceNo`不以C開頭的觀察值。  
但根據以上輸出結果，這些該選出來的觀察值的`UnitPrice`觀察值均為0，因此增加`UnitPrice`的條件進行驗證。

In [24]:
original_data[(original_data["Quantity"] < 0) & (original_data["InvoiceNo"].str[0] != "C") & (original_data["UnitPrice"] != 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [21]:
original_data[original_data["UnitPrice"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom


## 清理數據

### 保存數據